In [1]:
import pandas as pd
import statsmodels as sm
import os
from tabulate import tabulate
import matplotlib.pyplot as plt
plt.style.use("../assets/plot_styles.mplstyle")
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from collections import Counter
import ast
import sys
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import redis
import string
sys.path.append('../library')
from core import prettyPrint, getLengthOfDict, makeSingular, flattenWithGenerator
from midStats import linearModelGeneral
from plotting import loadPalette, loadTableStyles, createBoxplotWithTTests
import pandas as pd
import holoviews as hv
from sklearn.metrics.pairwise import cosine_similarity
from holoviews import opts, dim
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from typing import Dict, List
import nest_asyncio
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm
import requests
import mobypy as mbp
import nltk
from nltk.corpus import wordnet as wn

# Ensure you have the necessary NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')
from dotenv import load_dotenv
from nltk.corpus import wordnet
from ast import literal_eval
import gensim
load_dotenv()

colorPalette = loadPalette()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/seankortschot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/seankortschot/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/seankortschot/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('../data/topicModelling/GoogleNews-vectors-negative300.bin.gz', binary = True)

In [3]:
# This needs to be unambiguously about the thing
# For example: Spirit doesn't work for haunting as it will yield syns like zest and energy
# You can just include specific keywords here as well (e.g., godzilla); with the kwWeight they will yield high scores
class ConceptMap:
    """ Concept Map
        - Creates a weighted list of dictionaries that have the core concepts that we are looking for
    """
    def __init__(self, kwWeight):
        self.kwWeight = kwWeight
        self.conceptMap = []

    def create(self, concepts):
        for concept, words in concepts.items():
            self.conceptMap.append({(concept, self.kwWeight): [(word, self.kwWeight) for word in words]})

concepts = {
    'haunting': ['devil','spirit','specter', 'satan', 'underworld', 'exorcist','priest', 'ghost', 'curse'],
    'killer': ['murder', 'victim', 'sadistic', 'stalker', 'crime', 'detective', 'psychopath'],
    'isolation': ['house', 'abandoned', 'cabin', 'desolate', 'forest', 'alone','dessert','mountain'],
    'trauma': ['psychological', 'mystery','disturbing','atmosphere', 'moody', 'depression','sadness','turmoil'],
    'monster': ['beast', 'godzilla', 'shark', 'alien', 'werewolf', 'vampire', 'spider']
}

conceptMap = ConceptMap(kwWeight = 0.00001)
conceptMap.create(concepts)

In [16]:
!pip install bert-embedding

  Using cached bert_embedding-1.0.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached typing-3.6.6-py3-none-any.whl.metadata (1.8 kB)
  Using cached numpy-1.14.6.zip (4.9 MB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of bert-embedding to determine which version is compatible with other requirements. This could take a while.
  Using cached bert_embedding-1.0.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached bert_embedding-0.1.5-py3-none-any.whl.metadata (5.3 kB)
  Using cached bert_embedding-0.1.4-py3-none-any.whl.metadata (5.3 kB)
  Using cached bert_embedding-0.1.3-py3-none-any.whl.metadata (5.4 kB)
  Using cached bert_embedding-0.1.2-py3-none-any.whl.metadata (5.2 kB)
  Using cached bert_embedding-0.1.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached bert_embedding-0.1.0-py3-none-any.whl.metadata (1.8 kB)
INFO: pip is still looking at multiple versions of bert-embedding to determine which version is compatible with other requirements. This

In [13]:
import mxnet as mx
from bert_embedding import BertEmbedding

ModuleNotFoundError: No module named 'mxnet'

In [12]:
all_sentences = ['wow cool so cool', 'hey this is cool', 'i have sadness']
storage = ContextNeighborStorage(sentences=all_sentences, model=bert)
storage.process_sentences()
storage.build_search_index()

NameError: name 'all_sentences' is not defined

In [4]:
def findAdjacentTerms(term: str = ''):
    adjTerms = set()
    
    synsets = wn.synsets(term, pos=wn.NOUN)
    
    for synset in synsets:
        # Explore hypernyms (more general terms)
        for hypernym in synset.hypernyms():
            for hyponym in hypernym.hyponyms():
                for lemma in hyponym.lemmas():
                    adjTerms.add(lemma.name())
        
        # Explore hyponyms (more specific terms)
        for hyponym in synset.hyponyms():
            for lemma in hyponym.lemmas():
                adjTerms.add(lemma.name())

    return [t.replace('_',' ') for t in list(adjTerms)]

In [10]:
model.wv.most_similar(positive='farts')

AttributeError: 'KeyedVectors' object has no attribute 'wv'

In [42]:
def performInitialCheck(conceptMap: list = []):
    output = []
    for concept in conceptMap:
        conceptKws = list(flattenWithGenerator(concept.values()))
        nAdj = len(list(flattenWithGenerator([findAdjacentTerms(t[0]) for t in conceptKws])))
        output.append((list(concept.keys())[0][0], nAdj))

    print(tabulate(output, headers=['Term', 'Starting Adjacency']))

In [ ]:
performInitialCheck(conceptMap.conceptMap)

In [ ]:
# Apply nest_asyncio to allow nested event loops
def constructAdjacencyGraph(category: dict, curDepth: int, maxDepth: int, curCat: str = '', simThresh: float = .6, foundSyns: list = []):
    """ Construct Adjacency Graph
        - Creates a synGraph based on a conceptMap
    """
    if curDepth > maxDepth:
        return category
    
    keyDict = {}
    
    for key, rootSynList in category.items():
        rootSynDict = {}
        
        for rootSyn in rootSynList:
            # We only want to use our initial category title and the keywords that we pass in
            if curDepth < 2:
                curCat = key[0]

            if rootSyn[0] not in model:
                print(f"Error: {key[0]}-{rootSyn[0]} not in model")
                continue
            
            adjTerms = findAdjacentTerms(rootSyn[0])
            
            similarities = [model.similarity(rootSyn[0], s) if s in model else 0 for s in adjTerms]

            synsFiltered = [
                (s, 1/similarities[i]) for i, s in enumerate(adjTerms) if (
                    (s not in foundSyns) and
                    (s in model) and 
                    (similarities[i] > simThresh)
                )
            ]
            
            foundSyns += [s for s,_ in synsFiltered]
            rootSynDict[rootSyn] = synsFiltered

        keyDict[key] = rootSynDict

    # Recursively find synonyms for the next depth level
    for key in keyDict:
        for rootSyn in keyDict[key]:
            keyDict[key][rootSyn] = constructAdjacencyGraph(category = {rootSyn: keyDict[key][rootSyn]}, curDepth = curDepth + 1, maxDepth = maxDepth, curCat = curCat, simThresh = simThresh, foundSyns = foundSyns)[rootSyn]
    
    
    return keyDict

synGraph = {}

for conceptDict in tqdm(conceptMap.conceptMap):
    category = list(conceptDict.keys())[0][0]

    synGraph[category] = constructAdjacencyGraph(
        category  = conceptDict, 
        curDepth  = 0, 
        maxDepth  = 15, 
        curCat    = list(conceptDict.keys())[0], 
        simThresh = .4,
        foundSyns = []
    )

In [ ]:
# Calculate the length of all elements
lengths = [(k, getLengthOfDict(v)) for k,v in synGraph.items()]
print(tabulate(lengths, headers=['class','syns']))

In [59]:
def extractPairs(d, parent=None, value=None, allPairs=None):
    if allPairs is None:
        allPairs = []

    if isinstance(d, dict):
        for k, v in d.items():
            if parent is not None:
                allPairs.append({'source': parent, 'target': k, 'value': value})
            extractPairs(v, k, value if value is not None else parent, allPairs)
    elif isinstance(d, list):
        for item in d:
            allPairs.append({'source': parent, 'target': item, 'value': value})

    return allPairs

In [60]:
allPairs = extractPairs(synGraph)

for p in allPairs:
    if p['value'] == None:
        p['value'] = p['source']
    
    if not isinstance(p['value'], tuple):
        p['value'] = (p['value'], 30)
    if not isinstance(p['target'], tuple):
        p['target'] = (p['target'], 30)
    if not isinstance(p['source'], tuple):
        p['source'] = (p['source'], 30)

In [61]:
allPairsDf = pd.DataFrame.from_dict(allPairs)

allPairsDf.to_csv('../data/horrorTaxonomy.csv')

In [62]:
graphDf = allPairsDf.applymap(lambda x: x[0])


In [ ]:
import graphistry
graphistry.register(api=3, protocol = 'https', server='hub.graphistry.com', personal_key_id=os.getenv("GRAPHISTRY_KEY_ID"), personal_key_secret=os.getenv("GRAPHISTRY_KEY_SECRET"))
g = graphistry.edges(graphDf, 'source','target')
g.plot()

In [64]:
import networkx as nx
from networkx.algorithms.connectivity import local_edge_connectivity


# Create a sample graph
G = nx.Graph()
# G.add_edges_from([(1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (4,5),(5,6),(8,10)])



In [65]:
nxPairs = [(e['source'][0],e['target'][0], {'weight':e['target'][1]}) for e in allPairs]

In [66]:
G.add_edges_from(nxPairs)

In [68]:
nx.resistance_distance(G, 'spider','insect')

# similarity1 = nx.simrank_similarity(G, 'haunting', 'haunting')

In [93]:
def wordSimilarityAgainstList(targetWord: str, vectorizer, vSearchList, searchList):
    vTargetWord = vectorizer.transform([targetWord]).toarray()
    
    maxSim = 0
    maxWord = ''
    for word, v_word in zip(searchList, vSearchList):
        cosSim = cosine_similarity(vTargetWord, v_word)[0][0]
        if cosSim > maxSim:
            maxSim = cosSim
            maxWord = word
    return maxWord, maxSim


def stringSimilarity(searchString, referenceString, vectorizer):
    vector1 = vectorizer.fit_transform([searchString])
    vector2 = vectorizer.transform([referenceString])

    cosSim = cosine_similarity(vector1, vector2)[0][0]

    return cosSim

In [ ]:
### These should be left in the other
r9 = redis.Redis(
    host='127.0.0.1',
    port=6379,
    encoding="utf-8",
    decode_responses=True,
    db=9
)

keys = r9.keys('*')
plots = r9.mget(keys)

idPlotDict = dict(zip(keys,plots))
ks = ['Story','Plot','Summary','Premise','Synopsis','Narratives']
idPlotDictFiltered = {i:p for i,p in idPlotDict.items() if any(k in p for k in ks)}
print(f"WE FOUND: {len(idPlotDictFiltered)} PLOT SUMMARIES!")

In [95]:
topicDf = pd.read_csv('../data/tmdbDetails.csv')

topicDf.dropna(subset='genres',inplace=True)
topicDf = topicDf[topicDf['genres'].str.contains('Horror')]
topicDf.dropna(subset='overview',inplace=True)

In [96]:
topicDf['year'] = pd.to_datetime(topicDf['release_date']).dt.year
titlesAndYears = topicDf[['title','year']]
topicDf['plot'] = topicDf['imdb_id'].map(idPlotDictFiltered)
topicDf['allText'] = topicDf['plot'] + ' ' + topicDf['overview']
overviews = list(set(topicDf['plot'].values.tolist()))
translator = str.maketrans('', '', string.punctuation)

### CUTOFF BETWEEN MODULE 1 and MODULE 2

In [101]:
topicModels = pd.read_csv("../data/horrorTaxonomy.csv")

wordGroups = {}

for e in topicModels.to_dict('records'):
    valueLit = ast.literal_eval(e['value'])
    if valueLit not in wordGroups.keys():
        wordGroups[valueLit] = []
    
    wordGroups[valueLit].append(ast.literal_eval(e['target']))

for k,v in wordGroups.items():
    wordGroups[k] = list(set(v))

In [102]:
def predictCategory(plot, db: bool = False, title: str = ''):
    plotClean = plot.translate(translator).lower()
    plotSingular = makeSingular(plotClean)
    plotWords = set(plotSingular.split(' '))

    vectorizer = CountVectorizer(analyzer='char', ngram_range = (4,4))

    sims = {}
    for category, syns in wordGroups.items():

        # for word in plotWords --> This gives us exact matches with proximity weightings, so it's much more reliable
        catSum = sum([w[1] for w in set(syns) if w[0] in plotWords]) # change to character cosine sim
        weightedInt = catSum / len(syns)

        # This gives us imprecise matches, still useful e.g., vampiric vs. vampire, and it saves us having to lemmatize/stem
        synWords = [w[0] for w in syns]
        stringSim = stringSimilarity(searchString = plotSingular, referenceString = ' '.join(synWords), vectorizer = vectorizer)

        totalCatSim = stringSim + (( 3 * weightedInt ) / len(plotWords) )
        
        sims[category[0]] = totalCatSim
    
    category = max(sims, key=sims.get)
    
    if db:
        print('='*(len(title) + 12))
        print(f"SCORES FOR: {title}")
        print('-'*(len(title) + 12))
        print(tabulate(sims.items(), headers=['Category','Score']))
        print('='*(len(title) + 12))
        print('')

    return sims

In [ ]:
categories = []

for imdbId, plot in tqdm(idPlotDictFiltered.items()):
    category = predictCategory(plot)

    category['imdbId'] = imdbId

    categories.append(category)

catDf = pd.DataFrame.from_dict(categories)

catDf.set_index('imdbId', inplace=True, drop=True)
catDf.head()


scaler = MinMaxScaler()
catDfScaled = pd.DataFrame(scaler.fit_transform(catDf), columns = catDf.columns)
catDfScaled.index = catDf.index
catDfScaled['classification'] = catDfScaled.idxmax(axis=1)
catDfScaled.sort_values(by='haunting', ascending=False).head(10)

catDfScaled.groupby('classification').size()

In [ ]:
tmdbDfRaw = pd.read_csv('../data/tmdbDetails.csv')
idNameMap = tmdbDfRaw.set_index('imdb_id')['title'].to_dict()

catDfScaled['title'] = catDfScaled.index.map(idNameMap)

catDfScaled.dropna(subset='title', inplace=True)

In [ ]:
catDfScaled.sort_index(inplace=True)
len(catDfScaled)